<a href="https://colab.research.google.com/github/AzizJaser/NLP-text-Generate/blob/main/Generate_text_according_to_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

tokenizer = Tokenizer() # convert text to sequences of integers, where each integer represents a specific word

with open('/content/fa6.txt', 'r', encoding='utf-8') as file:
    data = file.read()



corpus = data.lower().split("\n") #The text data is converted to lowercase and split into lines. Each line is considered a separate entry in the corpus.

tokenizer.fit_on_texts(corpus) # tokenizer processes the corpus and assigns a unique integer ID to each distinct word in the corpus.
total_words = len(tokenizer.word_index) + 1






input_sequences = [] # Initialize an empty list to store input sequences
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0] # Convert each line to a sequence of tokens integers
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1] # Create n-gram sequences for each line
		input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences]) # Find the length of the longest sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')) # Pad all sequences to the length of the longest sequence

xs, labels = input_sequences[:,:-1],input_sequences[:,-1] # Separate the sequences into input data xs and labels last word of each sequence

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words) # One-hot encode the labels




model = Sequential() # Build the model

model.add(Embedding(total_words, 50, input_length=max_sequence_len-1)) # Add an Embedding layer (size of vocab,dimensionality of the embedding vectors,length of input)
model.add(Bidirectional(LSTM(150))) # Add a Bidirectional LSTM layer (numbers of units)
model.add(Dense(total_words, activation='softmax')) # Add a Dense layer with softmax activation for output (numbers of dimensionality optput, the activation fun)
adam = Adam(lr=0.01) # Initialize the Adam optimizer (ratio)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy']) # Compile the model
history = model.fit(xs, ys, epochs=30, verbose=1) # Train the model

print(model)



Epoch 1/30
2252/2252 [==============================] - 175s 73ms/step - loss: 7.5638 - accuracy: 0.1114
Epoch 2/30
2252/2252 [==============================] - 81s 36ms/step - loss: 6.5541 - accuracy: 0.1614
Epoch 3/30
2252/2252 [==============================] - 75s 33ms/step - loss: 5.8792 - accuracy: 0.1827
Epoch 4/30
2252/2252 [==============================] - 74s 33ms/step - loss: 5.2310 - accuracy: 0.2028
Epoch 5/30
2252/2252 [==============================] - 71s 31ms/step - loss: 4.6103 - accuracy: 0.2301
Epoch 6/30
2252/2252 [==============================] - 70s 31ms/step - loss: 4.0331 - accuracy: 0.2753
Epoch 7/30
2252/2252 [==============================] - 70s 31ms/step - loss: 3.5090 - accuracy: 0.3457
Epoch 8/30
2252/2252 [==============================] - 71s 31ms/step - loss: 3.0355 - accuracy: 0.4247
Epoch 9/30
2252/2252 [==============================] - 70s 31ms/step - loss: 2.6241 - accuracy: 0.4918
Epoch 10/30
2252/2252 [==============================] - 70s 31

In [ ]:
start_text = "عثمان" # Starting text for the sequence
next_words = 25 # Number of words to generate

for _ in range(next_words): # Loop to generate the specified number of words
	token_list = tokenizer.texts_to_sequences([start_text])[0] # Convert the current sequence of words to a list of token IDs
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre') # Pad the sequence to ensure it has a consistent length
	predicted = np.argmax(model.predict(token_list), axis=-1) # Use the model to predict the next word as a token ID
	output = ""  # Initialize a variable to hold the predicted word
	for word, index in tokenizer.word_index.items(): # Loop through the word index to find the word corresponding to the predicted token ID
		if index == predicted:
			output = word
			break
	start_text += " " + output # Add the predicted word to the current sequence
print(start_text)


1/1 [==============================] - 0s 26ms/step
عثمان بن عفان معه إمرأته رقية ابنة رسول الله صلى الله عليه وسلم، وأبو حذيفة بن عتبة بن عبد الله بن عمر بن مخزوم، فولدت له
